# Titanic

In [1]:
import os
import pandas as pd
import torch
from torch.utils.data import Dataset, DataLoader, random_split

In [2]:
class TitanicDataset(Dataset):
    def __init__(self, X, y):
        self.X = torch.FloatTensor(X)
        self.y = torch.LongTensor(y)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        feature = self.X[idx]
        target = self.y[idx]
        return {'input': feature, 'target': target}

    def __str__(self):
        str = "Data Size: {0}, Input Shape: {1}, Target Shape: {2}".format(
          len(self.X), self.X.shape, self.y.shape
        )
        return str

### Trainning Data Set
해당 클래스는 학습을 위한 Dataset 클래스입니다.

<맵버 변수>
- X : Input Data Tensor
- y : Target Data Tensor

<맵버 함수>
- len() : Input Data의 길이 출력
- getitem(idx) : idx에 해당하는 Input, Target 값을 사전형으로 리턴
- str() : 해당 Data Set의 정보 출력

In [3]:
class TitanicTestDataset(Dataset):
    def __init__(self, X):
        self.X = torch.FloatTensor(X)

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        feature = self.X[idx]
        return {'input': feature}

    def __str__(self):
        str = "Data Size: {0}, Input Shape: {1}".format(
          len(self.X), self.X.shape
        )
        return str

### Test Data Set
해당 클래스는 테스트를 위한 Dataset 클래스입니다.

<맵버 변수>
- X : Input Data Tensor

<맵버 함수>
- len() : Input Data의 길이 출력
- getitem(idx) : idx에 해당하는 Input 값을 사전형으로 리턴
- str() : 해당 Data Set의 정보 출력

In [4]:
def get_preprocessed_dataset():
    train_data_path = os.path.join(".", "Data", "train.csv")
    test_data_path = os.path.join(".", "Data", "test.csv")

    train_df = pd.read_csv(train_data_path)
    test_df = pd.read_csv(test_data_path)

    all_df = pd.concat([train_df, test_df], sort=False)

    all_df = get_preprocessed_dataset_1(all_df)

    all_df = get_preprocessed_dataset_2(all_df)

    all_df = get_preprocessed_dataset_3(all_df)

    all_df = get_preprocessed_dataset_4(all_df)

    all_df = get_preprocessed_dataset_5(all_df)

    all_df = get_preprocessed_dataset_6(all_df)

    train_X = all_df[~all_df["Survived"].isnull()].drop("Survived", axis=1).reset_index(drop=True)
    train_y = train_df["Survived"]

    test_X = all_df[all_df["Survived"].isnull()].drop("Survived", axis=1).reset_index(drop=True)

    dataset = TitanicDataset(train_X.values, train_y.values)
    #print(dataset)
    train_dataset, validation_dataset = random_split(dataset, [0.8, 0.2])
    test_dataset = TitanicTestDataset(test_X.values)
    #print(test_dataset)

    return train_dataset, validation_dataset, test_dataset

### 데이터 생성 함수
- train_data_path : 학습 데이터의 파일 경로
- test_data_path : 테스트 데이터의 파일 경로
- train_df : 가공되지 않은 순수 학습 데이터
    - [418 rows x 12 columns]의 2차원 DataFrame
    - rows : Data의 수
    - columns : PassengerId(승객 번호), Survived(생존여부), Pclass(티켓 클래스), Name(이름), Sex(성별), Age(나이), SibSp(동승한 자매/배우자의 수), Parch(동승한 부모/자식의 수), Ticket(티켓 번호), Fare(승객 요금), Cabin(방 호수), Embarked(탑승지)
- test_df : 가공되지 않은 순수 테스트 데이터, train_df와 동일(row의 수만 다름)
- all_df : 데이터 가공을 위해 train_df, test_df를 병합한 데이터, 추가 함수를 이용하여 데이터를 가공
- train_x : 가공된 데이터에 대하여 "survived"가 결측값이 아니라면(train_data) 해당 행을 삭제합니다. 즉 학습 데이터의 데이터 값만을 가집니다.
- train_y : 학습데이터의 "survived"의 값만을 가져옵니다. 즉 target값 만을 가집니다.
- dataset : 가공된 데이터를 통하여 만들어진 학습에 사용될 데이터
- train_dataset, validation_dataset : 학습에 사용될 데이터를 8 : 2로 랜덤 분할한 학습 데이터, 검증 데이터
- test_dataset : 가공된 데이터를 통해 만들어진 테스트 데이터

In [5]:
def get_preprocessed_dataset_1(all_df):
    # Pclass별 Fare 평균값을 사용하여 Fare 결측치 메우기
    Fare_mean = all_df[["Pclass", "Fare"]].groupby("Pclass").mean().reset_index()
    Fare_mean.columns = ["Pclass", "Fare_mean"]
    all_df = pd.merge(all_df, Fare_mean, on="Pclass", how="left")
    all_df.loc[(all_df["Fare"].isnull()), "Fare"] = all_df["Fare_mean"]

    return all_df

### 1차 데이터 가공
- Fare 데이터의 결측치를 Pclass의 평균으로 대체

In [6]:
def get_preprocessed_dataset_2(all_df):
    # name을 세 개의 컬럼으로 분리하여 다시 all_df에 합침
    name_df = all_df["Name"].str.split("[,.]", n=2, expand=True)
    name_df.columns = ["family_name", "honorific", "name"]
    name_df["family_name"] = name_df["family_name"].str.strip()
    name_df["honorific"] = name_df["honorific"].str.strip()
    name_df["name"] = name_df["name"].str.strip()
    all_df = pd.concat([all_df, name_df], axis=1)

    return all_df

### 2차 데이터 가공
- 이름에 대하여 family_name(성), honorific(경칭), name(이름)으로 구분 후 총 데이터에 해당 3 컬럼으로 변경

In [7]:
def get_preprocessed_dataset_3(all_df):
    # honorific별 Age 평균값을 사용하여 Age 결측치 메우기
    honorific_age_mean = all_df[["honorific", "Age"]].groupby("honorific").median().round().reset_index()
    honorific_age_mean.columns = ["honorific", "honorific_age_mean", ]
    all_df = pd.merge(all_df, honorific_age_mean, on="honorific", how="left")
    all_df.loc[(all_df["Age"].isnull()), "Age"] = all_df["honorific_age_mean"]
    all_df = all_df.drop(["honorific_age_mean"], axis=1)

    return all_df

### 3차 데이터 가공
- Age 데이터의 결측치를 honorific의 평균으로 대체

In [8]:
def get_preprocessed_dataset_4(all_df):
    # 가족수(family_num) 컬럼 새롭게 추가
    all_df["family_num"] = all_df["Parch"] + all_df["SibSp"]

    # 혼자탑승(alone) 컬럼 새롭게 추가
    all_df.loc[all_df["family_num"] == 0, "alone"] = 1
    all_df["alone"].fillna(0, inplace=True)

    # 학습에 불필요한 컬럼 제거
    all_df = all_df.drop(["PassengerId", "Name", "family_name", "name", "Ticket", "Cabin"], axis=1)

    return all_df

### 4차 데이터 가공
- SibSp(동승한 자매/배우자의 수), Parch(동승한 부모/자식의 수)의 데이터를 통하여 가족 수를 총 데이터에 추가
- 혼자 탑승자에 대한 컬럼을 따로 추가
- 학습에 불필요한 데이터 즉 생존에 연관 없는 데이터에 대한 삭제(승객 id, 이름, 성, 티켓 가격, 방 호수)

In [9]:
def get_preprocessed_dataset_5(all_df):
    # honorific 값 개수 줄이기
    all_df.loc[
    ~(
            (all_df["honorific"] == "Mr") |
            (all_df["honorific"] == "Miss") |
            (all_df["honorific"] == "Mrs") |
            (all_df["honorific"] == "Master")
    ),
    "honorific"
    ] = "other"
    all_df["Embarked"].fillna("missing", inplace=True)

    return all_df

### 5차 데이터 가공
- 경칭에 대하여 Mr, Miss, Mrs, Master를 제외하고는 other로 묶기
- 탑승지가 결측치인 경우 missing의 값으로 대체

In [10]:
def get_preprocessed_dataset_6(all_df):
    # 카테고리 변수를 LabelEncoder를 사용하여 수치값으로 변경하기
    category_features = all_df.columns[all_df.dtypes == "object"]
    from sklearn.preprocessing import LabelEncoder
    for category_feature in category_features:
        le = LabelEncoder()
        if all_df[category_feature].dtypes == "object":
            le = le.fit(all_df[category_feature])
            all_df[category_feature] = le.transform(all_df[category_feature])

    return all_df

### 6차 데이터 가공
- category_features : all_df에서 데이터 타입이 "object"인 열들의 이름이 담김 즉, 이는 카테고리 변수들의 이름을 모아놓은 것
- LabelEncoder :  각 카테고리를 고유한 정수값으로 매핑
- LabelEncoder를 사용하여 카테고리 변수들을 수치값으로 변경 
- le = le.fit(all_df[category_feature])를 통해 해당 카테고리 변수의 고유한 값들을 학습
- all_df[category_feature] = le.transform(all_df[category_feature])를 통해 해당 카테고리 변수를 수치값으로 변환

# Titanic Trainning

In [16]:
import wandb
from torch import nn
from datetime import datetime

In [17]:
wandb.login()

True

In [18]:
class MyModel(nn.Module):
    def __init__(self, n_input, n_output):
        super().__init__()

        self.model = nn.Sequential(
          nn.Linear(n_input, 30),
          nn.ReLU(),
          nn.Linear(30, 30),
          nn.ReLU(),
          nn.Linear(30, n_output),
        )

    def forward(self, x):
        x = self.model(x)
        return x

### Model
- 입력 layer : intput by 30
- 히든 layer : 30 by 30
- 출력 layer : 30 by output

- 활성화 함수
    - 입력 -> 히든1(ReLu)
    - 히든1 -> 출력(ReLu)

In [19]:
def test(test_data_loader):
    print("[TEST]")
    batch = next(iter(test_data_loader))
    print("{0}".format(batch['input'].shape))
    my_model = MyModel(n_input=11, n_output=2)
    output_batch = my_model(batch['input'])
    prediction_batch = torch.argmax(output_batch, dim=1)
    for idx, prediction in enumerate(prediction_batch, start=892):
        print(idx, prediction.item())

### test
- batch : test_dasta_loader.shape -> torch.Size([418, 11])
- my_model : 입력 : 11, 출력 : 2
- output_batch : output_batch.shape -> ([418, 2])
- prediction_batch : output_batch.shape -> (418)

In [20]:
if __name__ == "__main__":
    ENV_NAME = "Kaggle_Titanic_FCN"
    use_wandb = True
    current_time = datetime.now().astimezone().strftime('%Y-%m-%d_%H-%M-%S')
    config = {
        "env_name": ENV_NAME,
        "max_num_epoch": 100,
        "batch_size": 16,
        "learning_rate": 0.001,
    }
    
    if use_wandb:
            wandb = wandb.init(
                project=config["env_name"],
                name=current_time,
                config=config
            )
    
    train_dataset, validation_dataset, test_dataset = get_preprocessed_dataset()

    print("train_dataset: {0}, validation_dataset.shape: {1}, test_dataset: {2}".format(
        len(train_dataset), len(validation_dataset), len(test_dataset)
    ))
    print("#" * 50, 1)

    for idx, sample in enumerate(train_dataset):
        print("{0} - {1}: {2}".format(idx, sample['input'], sample['target']))

    print("#" * 50, 2)

    train_data_loader = DataLoader(dataset=train_dataset, batch_size=16, shuffle=True)
    validation_data_loader = DataLoader(dataset=validation_dataset, batch_size=16, shuffle=True)
    test_data_loader = DataLoader(dataset=test_dataset, batch_size=len(test_dataset))

    print("[TRAIN]")
    for idx, batch in enumerate(train_data_loader):
        print("{0} - {1}: {2}".format(idx, batch['input'].shape, batch['target'].shape))
        
    print("[VALIDATION]")
    for idx, batch in enumerate(validation_data_loader):
        print("{0} - {1}: {2}".format(idx, batch['input'].shape, batch['target'].shape))

    print("#" * 50, 3)

    test(test_data_loader)
        
    my_model = MyModel(n_input=11, n_output=2)
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(my_model.parameters(), lr=config["learning_rate"])

    for epoch in range(config["max_num_epoch"]):
        my_model.train()
        total_loss = 0.0
        correct = 0
        total = 0

        for idx, batch in enumerate(train_data_loader):
            optimizer.zero_grad()
            input_data = batch['input']
            target = batch['target']

            output = my_model(input_data)
            loss = criterion(output, target)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()

            _, predicted = torch.max(output.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()

        accuracy = correct / total
        average_loss = total_loss / len(train_data_loader)

        if use_wandb:
            wandb.log({"train_loss": average_loss, "train_accuracy": accuracy})

        print(f"Epoch [{epoch + 1}/{config['max_num_epoch']}], "
            f"Loss: {average_loss:.4f}, "
            f"Accuracy: {accuracy:.4f}")
        
        my_model.eval()
        total_loss = 0.0
        correct = 0
        total = 0

        with torch.no_grad():
            for idx, batch in enumerate(validation_data_loader):
                input_data = batch['input']
                target = batch['target']

                output = my_model(input_data)
                loss = criterion(output, target)

                total_loss += loss.item()

                _, predicted = torch.max(output.data, 1)
                total += target.size(0)
                correct += (predicted == target).sum().item()

        accuracy = correct / total
        average_loss = total_loss / len(validation_data_loader)

        if use_wandb:
            wandb.log({"val_loss": average_loss, "val_accuracy": accuracy})

        print(f"Validation Loss: {average_loss:.4f}, "
            f"Validation Accuracy: {accuracy:.4f}")
    wandb.finish()

        
    final_model_path = './Model/my_model_final.pth'
    torch.save(my_model.state_dict(), final_model_path)

    my_model = MyModel(n_input=11, n_output=2)
    my_model.load_state_dict(torch.load('./Model/my_model_final.pth'))
    my_model.eval()

    # 테스트 데이터를 DataLoader에 넣습니다.
    test_data_loader = DataLoader(dataset=test_dataset, batch_size=len(test_dataset))

    # 모델을 사용하여 테스트 데이터에 대한 예측을 수행합니다.
    predictions = []
    for batch in test_data_loader:
        inputs = batch['input']
        outputs = my_model(inputs)
        predictions_batch = torch.argmax(outputs, dim=1)
        predictions.extend(predictions_batch.tolist())

    # submission.csv 파일을 생성합니다.
    submission_df = pd.DataFrame({
        'PassengerId': range(892, 892 + len(predictions)),
        'Survived': predictions
    })

    # CSV 파일로 저장합니다.
    submission_df.to_csv('./Out/submission.csv', index=False)

    print("Submission 파일 'submission.csv'가 생성되었습니다.")

train_dataset: 713, validation_dataset.shape: 178, test_dataset: 418
################################################## 1
0 - tensor([ 3.0000,  0.0000, 22.0000,  1.0000,  0.0000, 14.4542,  0.0000, 13.3029,
         1.0000,  1.0000,  0.0000]): 0
1 - tensor([ 1.0000,  1.0000, 29.0000,  0.0000,  0.0000,  0.0000,  2.0000, 87.5090,
         2.0000,  0.0000,  1.0000]): 0
2 - tensor([ 2.0000,  1.0000, 25.0000,  0.0000,  0.0000, 13.0000,  2.0000, 21.1792,
         2.0000,  0.0000,  1.0000]): 0
3 - tensor([ 3.0000,  1.0000, 29.0000,  0.0000,  0.0000,  7.2250,  0.0000, 13.3029,
         2.0000,  0.0000,  1.0000]): 0
4 - tensor([ 3.0000,  1.0000, 22.0000,  0.0000,  0.0000,  7.2500,  2.0000, 13.3029,
         2.0000,  0.0000,  1.0000]): 0
5 - tensor([ 3.0000,  1.0000, 50.0000,  0.0000,  0.0000,  8.0500,  2.0000, 13.3029,
         2.0000,  0.0000,  1.0000]): 0
6 - tensor([ 2.0000,  1.0000, 23.0000,  0.0000,  0.0000, 10.5000,  2.0000, 21.1792,
         4.0000,  0.0000,  1.0000]): 0
7 - tensor([ 1.00

torch.Size([418, 11])
892 0
893 0
894 0
895 0
896 0
897 0
898 0
899 0
900 0
901 0
902 0
903 1
904 1
905 0
906 1
907 0
908 0
909 0
910 0
911 0
912 1
913 0
914 1
915 1
916 0
917 0
918 1
919 0
920 1
921 0
922 0
923 0
924 0
925 0
926 1
927 0
928 0
929 0
930 0
931 0
932 0
933 1
934 0
935 0
936 1
937 0
938 1
939 0
940 0
941 0
942 1
943 0
944 0
945 0
946 0
947 0
948 0
949 0
950 0
951 0
952 0
953 0
954 0
955 0
956 0
957 0
958 0
959 1
960 1
961 0
962 0
963 0
964 0
965 1
966 0
967 0
968 0
969 1
970 0
971 0
972 1
973 0
974 1
975 0
976 0
977 0
978 0
979 0
980 0
981 1
982 0
983 0
984 1
985 0
986 1
987 0
988 0
989 0
990 0
991 0
992 1
993 0
994 0
995 0
996 0
997 0
998 0
999 0
1000 0
1001 0
1002 0
1003 0
1004 1
1005 0
1006 0
1007 0
1008 0
1009 1
1010 1
1011 0
1012 1
1013 0
1014 1
1015 0
1016 0
1017 0
1018 0
1019 0
1020 0
1021 0
1022 0
1023 1
1024 0
1025 0
1026 0
1027 0
1028 0
1029 0
1030 0
1031 0
1032 0
1033 0
1034 0
1035 0
1036 1
1037 0
1038 1
1039 0
1040 1
1041 0
1042 1
1043 0
1044 0
1045 0
1046 0
1

Epoch [55/100], Loss: 0.4350, Accuracy: 0.8163
Validation Loss: 0.4026, Validation Accuracy: 0.8258
Epoch [56/100], Loss: 0.4329, Accuracy: 0.8065
Validation Loss: 0.3937, Validation Accuracy: 0.8202
Epoch [57/100], Loss: 0.4488, Accuracy: 0.8121
Validation Loss: 0.4234, Validation Accuracy: 0.7978
Epoch [58/100], Loss: 0.4374, Accuracy: 0.7938
Validation Loss: 0.4288, Validation Accuracy: 0.8146
Epoch [59/100], Loss: 0.4202, Accuracy: 0.8135
Validation Loss: 0.4170, Validation Accuracy: 0.7978
Epoch [60/100], Loss: 0.4195, Accuracy: 0.8149
Validation Loss: 0.3884, Validation Accuracy: 0.8202
Epoch [61/100], Loss: 0.4164, Accuracy: 0.8191
Validation Loss: 0.4440, Validation Accuracy: 0.7865
Epoch [62/100], Loss: 0.4367, Accuracy: 0.8149
Validation Loss: 0.4457, Validation Accuracy: 0.8090
Epoch [63/100], Loss: 0.4149, Accuracy: 0.8065
Validation Loss: 0.4897, Validation Accuracy: 0.8090
Epoch [64/100], Loss: 0.4370, Accuracy: 0.8107
Validation Loss: 0.4250, Validation Accuracy: 0.8090


train_accuracy,▁▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇█▇█▇▇███▇█████████████
train_loss,█▃▃▃▂▂▂▂▂▂▂▂▁▁▂▁▁▂▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_accuracy,▁▃▂▂▃▂▆▅▆▅▅▆▅▇▆▆▇▆▇▆▇▇▇▆▆▇██▆█▇▇▇▇█▇▅▇██
val_loss,▇█▆▆▅▆▄▄▆▅▅▂▃▂▂▃▂▃▂▃▁▂▁▂▃▂▁▁▄▄▂▂▁▁▂▁▃▂▅▃
train_accuracy,0.82048
train_loss,0.43231
val_accuracy,0.83146
val_loss,0.43672


Submission 파일 'submission.csv'가 생성되었습니다.


![img](./Img/kaggle_titanic_res_2023-10-03.png)